## 1. Import

In [22]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import random
from connect_four import ConnectFour
from random_bot import RandomBot

## 2. NN Bot Class

In [24]:
class NeuralNetworkBot:
    def __init__(self, piece, model=None):
        self.piece = piece
        self.model = model or self.build_model()

    def build_model(self):
        model = Sequential([
            Dense(64, input_dim=7 * 6, activation='relu'),
            Dense(64, activation='relu'),
            Dense(7, activation='linear')
        ])
        model.compile(optimizer=Adam(), loss='mse')
        return model
    
    def convert_board(self, board):
        piece_to_number = {' ': 0, 'X': 1, 'O': -1}
        return np.array([piece_to_number[cell] for cell in board]).reshape(1, -1)

    def choose_column(self, game):
        state = self.convert_board(game.board)
        q_values = self.model.predict(state, verbose=0)
        available_columns = [c for c in range(game.width) if game.board[c] == ' ']
        q_values[0, [c for c in range(game.width) if c not in available_columns]] = -np.inf
        return np.argmax(q_values)
    
    def mutate(self, mutation_rate=0.01):
        weights = self.model.get_weights()
        new_weights = []
        for weight in weights:
            if np.ndim(weight) == 1:
                new_weights.append(weight + mutation_rate * np.random.randn(*weight.shape))
            else:
                new_weights.append(weight + mutation_rate * np.random.randn(*weight.shape))
        self.model.set_weights(new_weights)

    def crossover(self, other):
        new_model = self.build_model()
        new_weights = []
        weights1 = self.model.get_weights()
        weights2 = other.model.get_weights()
        for w1, w2 in zip(weights1, weights2):
            mask = np.random.rand(*w1.shape) > 0.5
            new_weights.append(np.where(mask, w1, w2))
        new_model.set_weights(new_weights)
        return NeuralNetworkBot(self.piece, new_model)

## Functions

In [3]:
def initialize_population(population_size, piece):
    return [NeuralNetworkBot(piece) for _ in range(population_size)]

In [5]:
def evaluate_fitness(bot, opponent, games=10):
    fitness = 0
    for _ in range(games):
        game = ConnectFour()
        while game.state == 'UNFINISHED':
            col = bot.choose_column(game)
            game.insert(col, bot.piece)
            if game.state != 'UNFINISHED':
                fitness += 1 if game.state == bot.piece + '_WON' else 0
                break

            col = opponent.choose_column(game)
            game.insert(col, opponent.piece)
            if game.state != 'UNFINISHED':
                fitness += -1 if game.state == opponent.piece + '_WON' else 0
                break
    return fitness

In [6]:
def select_top_performers(population, fitnesses, num_top):
    sorted_indices = np.argsort(fitnesses)[::-1]
    return [population[i] for i in sorted_indices[:num_top]]

In [7]:
def generate_new_population(top_performers, population_size, mutation_rate=0.01):
    new_population = []
    while len(new_population) < population_size:
        parent1, parent2 = random.sample(top_performers, 2)
        child = parent1.crossover(parent2)
        child.mutate(mutation_rate)
        new_population.append(child)
    return new_population

In [9]:
def train_evolutionary(population_size=50, generations=100, num_top=10, mutation_rate=0.01, games_per_fitness=10):
    population = initialize_population(population_size, 'X')
    opponent = RandomBot('O')

    for generation in range(generations):
        fitnesses = [evaluate_fitness(bot, opponent, games_per_fitness) for bot in population]
        top_performers = select_top_performers(population, fitnesses, num_top)
        population = generate_new_population(top_performers, population_size, mutation_rate)
        print(f'Generation {generation + 1}, Best Fitness: {max(fitnesses)}')

    return population

In [25]:
trained_population = train_evolutionary()

Generation 1, Best Fitness: 10
